# AI 101 - Lab 1
## Deploy a local Jupyter Notebook cluster on HPE Ezmeral Container Platform

### **Lab workflow:**

In this lab:

1. As tenant user, you will deploy a local Jupyter Notebook application KubeDirector cluster on HPE Ezmeral Container Platform and use it as a code sandbox to get started with Deep Learning Neural Network model.

2. You will then access your local Jupyter Notebook web UI via the gateway network service port to run the AI 101 hands-on workshop and dive into the typical data scientist work to devlop a deep learning model.

#### Initialize the environment:

Let's first define the environment variables needed to execute this lab part.

In [ ]:
#
# environment variables to be verified by the student
#
studentId="student{{ STDID }}" # your Jupyter Notebook student Identifier (i.e.: student<xx>)
username="student{{ STDID }}" # your HPE ECP tenant login credentials - username 
password="{{ PASSSTU }}" # your HPE ECP tenant login credentials - password - Please replace with password provided in your registration

tenantname="K8sMLTenant" #case sensitive
#
# fixed environment variables setup by the HPE ECP lab administrator - Please DO NOT MODIFY!!
#
gateway_host="{{ HPEECPGWNAME }}"
controller_endpoint="${gateway_host}:8080"
Internet_access="{{ JPHOSTEXT }}"

gateway_host="gateway2.etc.fr.comm.hpecorp.net"
controller_endpoint="${gateway_host}:8080"
Internet_access="notebooks.hpedev.io"

clusterName="jupyter-notebook-${studentId}"
JupyterNotebookApp="cr-cluster-jupyter-notebook.yaml" # the Jupyter Notebook KD App manifest you will deploy to build your model
sc_secret="sc-secret-AI101-students.yaml"

echo "your operation context is:" $username "on tenant" $tenantname 
echo "your REST API endpoint is:" $controller_endpoint

#### Authenticate as a tenant user in the specified tenant:

In [ ]:
sessionlocation=$(curl -k -i -s --request POST "https://${controller_endpoint}/api/v2/session" \
--header 'Accept: application/json' \
--header 'Content-Type: application/json' \
--data-raw '{
"name": "'"$username"'",
"password": "'"$password"'",
"tenant_name": "'"$tenantname"'"
}' | grep Location | awk '{print $2}' | tr -d '\r') #we remove any cr that might exist
echo "This is your session location: " $sessionlocation
SessionId=$(echo $sessionlocation | cut -d'/' -f 5) # extract sessionId for later, for logout
echo "This is your session_Id:" $SessionId

#### Get the Kubeconfig file for your tenant working context:

In [ ]:
mkdir -p ~/.kube
ls ~/.kube
rm -f ~/.kube/config
curl -k -s --request GET "https://${controller_endpoint}/api/v2/k8skubeconfig" \
--header "X-BDS-SESSION: $sessionlocation" \
--header 'Accept: application/json' \
--header 'Content-Type: application/json' > ~/.kube/config
#
echo "Kubeconfig file fetched. Your current Kubernetes working context is: "
kubectl config current-context

## Deploying your local Jupyter Notebook cluster

#### Deploy an instance of the Jupyter Notebook KubeDirector application:
You will deploy an instance of the **jupyter-notebook** KubeDirector application (kdapp) with the Training cluster _Connection_ by creating a KubeDirector virtual cluster (kdcluster). A kdcluster identifies the desired kdapp and specifies runtime configuration parameters, such as the size and resource requirements of the virtual cluster. 

> **Note:** _The Jupyter-Notebook kdapp includes the open source Machine learning toolkits, software libraries and frameworks for developing and training models such as TensorFlow, scikit-learn, keras, XGBoost, matplotlib, Jupyter Notebook, Numpy, Scipy, Pandas..._

Like any other containerized application deployment on Kubernetes, the K8s API call requires the kubectl operation type (create or apply) and the application manifest (a YAML file that describes the attributes of the application).  

As you are all sharing the same tenant context and Kubernetes cluster resources, let's make sure your application deployment name will be unique among the tenant users. Here we replace the string "example" with your "studentId" in the application manifest file.

> **Note:** _one of the most interesting parts of the kdcluster spec is the **Connections** stanza (a related group of attributes), which identifies other resources of interest to that kdcluster. Here, you attach your local Jupyter Notebook cluster to two secrets: 
a secret that identifies the values for the source control repository from which to pull files (that is the AI-101 notebook and associated files) when attached to the local Jupyter Notebook cluster. 
Another secret that contains the values for external Ezmeral Container Platform's LDAP/AD authentication._ 

In [ ]:
kubectl apply -f $sc_secret
kubectl apply -f $JupyterNotebookApp

After a few seconds, you should get the response message to your K8s API request: *kubedirectorcluster/Your-instance-name created*.  

In the next steps, you will use kubectl commands in the context of your tenant user account and get the Notebook service endpoint with token-based authorization to connect to it.

## Check the deployment status of the local Jupyter Notebook cluster

After creating the instance of the KubeDirector Application, you can use the `kubectl describe kdcluster` command below to observe its status and the standard Kubernetes resources that compose the application virtual cluster (statefulsets, pods, services, persistent volume claim if any), as well as any events logged against it.

The virtual cluster status indicates its overall "state" (top-level property of the status object). It should have a value of **"configured"**. 

> **Note:** _The first time a virtual cluster of a given KubeDirector Application type is created, it may take some minutes to reach its **"configured"** state, as the relevant Docker image must be downloaded and imported._ 

**Repeat the command below until the kdcluster is in state "configured"**

In [ ]:
kubectl describe kdcluster $clusterName

## Get the local Jupyter Notebook service endpoint

In [ ]:
#
# Getting the service endpoint URL:
#
JupyterAppURL=$(kubectl describe service -l  kubedirector.hpe.com/kdcluster=${clusterName} | grep gateway/8000 | awk '{print $2}')
JupyterAppPort=$(echo $JupyterAppURL | cut -d':' -f 2) # extract the gateway re-mapped port value.
myJupyterApp_endpoint="https://$gateway_host:$JupyterAppPort"
#echo "Your application service endpoint re-mapped port is: "$JupyterAppPort
#echo "Your Intranet application service endpoint is: "$myJupyterApp_endpoint
echo "Your Jupyter Notebook service endpoint URL is: https://"$Internet_access:$JupyterAppPort
#

## Connect to your local Jupyter Notebook web UI
Click the service endpoint URL above to connect to your Jupyter Notebook server.
This opens a Jupyter Notebook login screen in a new browser tab. Login using your jupyterHub account username and password provided to you in the registration email. And enjoy the AI 101 hands-on workshop!

### **Time to go through some cleanup**

Delete your secret and your local Jupyter Notebook cluster

In [ ]:
kubectl delete -f $sc_secret
kubectl delete -f $JupyterNotebookApp

In [ ]:
curl -k -i -s --request DELETE "https://${controller_endpoint}/api/v2/session/${SessionId}" \
--header "X-BDS-SESSION: $sessionlocation" \
--header 'Accept: application/json' \
--header 'Content-Type: application/json'

Thank you!!!